In [2]:
#Topics covered
#Memory references
#What variables really are 
# memory management
# reference counting and garbage collection
#Dynamic and static typing
#Mutability and immutability
# Shared references!
#Variable equality
#Everything is an object including Classes themselves!

In [3]:
#Variables and Memory references!

In [5]:
# Variable are Memory references!
#Check more on heap memory!
# Storing and retrieving objects from heap is taken care of 
#by the python memory manager!

In [9]:
my_var1 = 10
#creates an object in memory  at address say 1000 and stores it there!

In [10]:
# my_var1 is a reference to 10, it is a pointer aka memory address of 10

In [11]:
my_var2 = 'hello'

In [12]:
#Variables in python are always references!

In [13]:
# In python we can find out the memory address referenced by a variable by using the id()
# function!

In [14]:
# id function will return a base 10 , and we can convert it into hex

In [28]:
#eg:
#a = 10
#print(hex(id(a)))

In [29]:
my_var = 10

In [30]:
print(my_var)

10


In [31]:
print(id(my_var))

1368812896


In [33]:
print(hex(id(my_var)))  # hexadecimal version of the address!

0x51966d60


In [34]:
greeting  = 'Hellooo'
print(greeting)

Hellooo


In [38]:
print(id(greeting))

2434899065968


In [39]:
print(hex(id(greeting)))

0x236eb4b3c70


### Reference Counting!

In [43]:
my_var = 11
# my_var is the reference/pointer to the object 11 created in the memory!

In [45]:
# How many variables are pointing to the same object?
# Here its 1 because no other variables refer to it !

In [49]:
other_var = my_var
# we are dealing with pointers!
#Now other_var still points to my_var
#Reference is shared, therefore reference count is 2

In [50]:
# if  no references are there
# Then the object is thrown away!

In [53]:
# Finding the Reference Count!
# sys.getrefcount (  <pass a variable> )
# passing my_var to getrefcount() creates an extra reference!

In [54]:
# There is another function that we can use without extra counting

In [55]:
# ctypes.c_long.from_address(address).value
# here we just pass the memory address, as we dont pass value!

In [56]:
import sys

In [64]:
a = [1, 2, 3, 4, 5, 6, 8, 9, 10 ]

In [65]:
id(a)

2434899180744

In [68]:
sys.getrefcount(a)
# See ref count is 2!  because the passing of  variable "a" caused this!

2

In [69]:
import ctypes

In [70]:
def ref_count(address : int):
    return ctypes.c_long.from_address(address).value

In [74]:
ref_count(id(a)) # we have to pass the address!
# But we still passed "a" inside the function
#id(a) is getting evaulated first
# and as id finishes running, IT RETURNS THE ADDRESS! NOT THE VARIABLE!
#SO BY THE TIME WE HAVE FINISHED ID , IT AS RELEASED ITS POINTER

1

In [75]:
# Its the same thing as doing this!

In [76]:
ref_count(2434899180744)

1

In [78]:
b = a# shared reference!

In [81]:
id(b) # same as "a"

2434899180744

In [83]:
ref_count(id(a))

2

In [92]:
c = a

In [93]:
ref_count(id(a))

2

In [94]:
c = 10

In [95]:
ref_count(id(a))  # Dropping "c"

1

In [88]:
b = None
ref_count(id(a)) # "b" is dropped

1

In [90]:
print(id(b)) # because 'b' i s None

1368368272


In [102]:
a_id = id(a)
a = None

ref_count(a_id)  # Whyt is this happening?

24408

In [100]:
ref_count(a_id)  # Whyt is this happening?

-420341279

In [103]:
ref_count(a_id)  # Whyt is this happening?

24407

In [104]:
# when last ref to that memory address is deleted ,
# memory manager frees up that address and something random pops up here!

### Garbage Collection

In [306]:
#As soon ref count is 0  python memory manager destroys object and reclaims the memory

In [307]:
# but its not so with circular references!
#circular references will lead to memory leak!

In [308]:
# Thats where garbage collector comes in 
# It will be able it identify circular references and delete it!

In [309]:
# It can be controlled programmatically using gc module!
# by default it is turned on
# you may turn it off if you're sure your code does not create circular references
# -but beware!
#be careful , this is more advanced!
#runs periodically on its own (if turned on)
# you can call it manually, and even do your own clean up!

In [310]:
#In general it works just fine 
# but not always
# especially  for Python <3.4
# even if one of the objects in circular references has a destructor , __del__()
# Then the order of objects may be important!
#but GC does not know what the order should be 
# So its marked as uncollectable!
#We would end up with a memory leak!

In [311]:
import ctypes
import gc  # garbage collection module

In [312]:
def ref_count(address):
    return ctypes.c_long.from_address(address).value

In [313]:
def object_by_id(object_id):
    for obj in gc.get_objects(): # every object in gc!
        if id(obj) == object_id:
            return "object exists"
        return " Not Found"

In [314]:
class A:
    def __init__(self):
        self.b = B(self)  #self.b is an instance of B()
        print('A: self: {0}, b: {1}'.format(hex(id(self)) ,hex(id(self.b))))

In [315]:
class B:
    def __init__(self, a):
        self.a = a
        print('B: self: {0}, a: {1}'.format(hex(id(self)) ,hex(id(self.a))))
        #print(self.a)

In [316]:
#Circular reference complete!


In [317]:
gc.disable() # disabled garbage collector!

In [318]:
my_var = A()
#my_var is passed into Self( of A)
#my_var.b is an instance of Class B which has my_var as a parameter

B: self: 0x236eb5d2cc0, a: 0x236eb5d2c50
A: self: 0x236eb5d2c50, b: 0x236eb5d2cc0


In [319]:
hex(id(my_var))

'0x236eb5d2c50'

In [320]:
print(hex(id(my_var.b)))

0x236eb5d2cc0


In [321]:
print(hex(id(my_var.b.a)))

0x236eb5d2c50


In [322]:
a_id = id(my_var)
b_id = id(my_var.b)

In [323]:
print(hex(id(a_id)))
print(hex(id(b_id)))

0x236eb60e150
0x236eb5c3d50


In [324]:
ref_count(a_id)

2

In [325]:
ref_count(b_id)

1

In [326]:
object_by_id(a_id)

' Not Found'

In [327]:
object_by_id(b_id)

' Not Found'

In [330]:
my_var = None

In [331]:
ref_count(a_id)

1

In [332]:
ref_count(b_id)

1

In [335]:
gc.collect()

0

In [337]:
object_by_id(a_id)

' Not Found'

In [338]:
object_by_id(b_id)

' Not Found'

In [339]:
ref_count(a_id)

-1

In [341]:
ref_count(b_id)

-1

In [344]:
gc.enable()

In [345]:
gc.isenabled()

True

## Dynamic Typing vs Static Typing

In [346]:
#Some languages (Java, C++ ...) are statically typed

In [348]:
#String     myvar = 'hello'
#datatype   Variable  value     

In [349]:
#Data type is associated with the variable name!!

In [350]:
#i.e
# myvar = 10 does not work!

In [351]:
# This is not so in Python
# It is dynamically typed!

In [352]:
#my var is purely a reference to a string object with value hello
# No type is "attached to my_var"

In [354]:
my_var = 10
print(my_var)

10


In [356]:
my_var = 'hello'
print(my_var)

hello


In [357]:
# Did type of my var change?
# NO  my var did not even have a type to begin with!

In [358]:
# We can use type() to find the type of the object pointed at any point in the 
# execution

In [359]:
a = 'String'

In [360]:
type(a)

str

In [361]:
a = 10

In [362]:
type(a)

int

In [363]:
a = lambda x: x**2

In [367]:
type(a)

function

In [368]:
a = 3 + 4j

In [369]:
type(a)

complex

In [370]:
# Everytime we change the referance of object!

### Variable reassignment

In [375]:
a = 10

In [376]:
a = 15 
# We are not changing the value inside the object 10
# But instead a new object of 15 is created 
# And the reference is changed!

In [377]:
# Infact content of integets can never be changed!

In [378]:
hex(id(a))

'0x51966e00'

In [379]:
type(a)

int

In [380]:
a = 15

In [382]:
hex(id(a)) 
#Memory address of a has changed!

'0x51966e00'

In [383]:
print(type(a))

<class 'int'>


In [385]:
a = a +1
hex(id(a)) #Memory address changed again!!

'0x51966e40'

In [386]:
a = 10
b = 10


In [387]:
hex(id(a)) 


'0x51966d60'

In [388]:
hex(id(b)) 

'0x51966d60'

In [390]:
# WHY ARE "a" and "b" are pointing to the same object!???
# Why is python doing this??

### Object mutability

In [391]:
#Object has a 
    # Type
    # and state / data
    
# Changing the data inside the object is called modifying the internal state of the object

In [3]:
#Eg:  say a variable called my_account of a datatype BankAccount
# Bank account has two properties
# Account no say 123456
# Balance : 150

#Now if  we modify the Balance to 500
# We say that the internal state(data) has changed


# BUT THE MEMORY ADDRESS HAS NOT CHANGED

#Now we say that the object has MUTATED

In [4]:
# An object whose internal state can be changed is called Mutable


In [ ]:
# An object whose internal state cannot

In [6]:
#Immutable objects
# Numbers (int, float, Booleans, etc...)
# Strings
# Tuples
# Frozen Sets
# User Defined Class

In [8]:
# Mutable types
# Lists (We can Add remove and replace)
#Sets 
#Dictionaries 
#User-Defined Classes

A warning about mutability


In [9]:
t = (1,2,3)  # tuples are immutable:
#elements cannot be deleted, inserted or replaced
# and also elements inside tuple are integers which are immutable too

In [14]:
# Now consider this
a = [1, 2]
b = [3, 4]
t = ( a, b)  # Now lists are mutable 
t

([1, 2], [3, 4])

In [15]:
a.append(100)
b.append(500)

In [20]:
 t 
# Now we can add elements to "a" and "b"
# But the tuple in itself is immutable !
#t is mutable but element inside t is mutable
# But t  changed

#Not to confuse mutability with completely unchanged

#It's not absolutely fixed

([1, 2, 100], [3, 4, 500])

In [21]:
my_list = [1, 2, 3]

type(my_list)

list

In [22]:
hex(id(my_list))

'0x190ead502c8'

In [23]:
my_list.append(4)

In [24]:
my_list


[1, 2, 3, 4]

In [27]:
hex(id(my_list))  # Memory address is not changed even after appending!

'0x190ead502c8'

In [37]:
my_list1 = [1, 2, 3]

In [40]:
hex(id(my_list1)) # Even though my_list and my_List1 contains the same values they are 
# not the same objects

'0x190eadb4f88'

In [41]:
my_list1 = my_list1 + [4]

In [42]:
my_list1

[1, 2, 3, 4]

In [43]:
# We did'nt append here, we concatenate two objects here!
# python combined two objects and returned a new one!

In [45]:
hex(id(my_list1)) # The id is  not the same because its another object

'0x190eaddd848'

In [46]:
my_dict = dict(key1 =1,key2 = 5)

In [47]:
my_dict

{'key1': 1, 'key2': 5}

In [48]:
hex(id(my_dict))

'0x190eaddaab0'

In [51]:
my_dict['key3'] = 10.2531

In [52]:
my_dict


{'key1': 1, 'key2': 5, 'key3': 10.2531}

In [54]:
hex(id(my_dict)) # The address has not changed

'0x190eaddaab0'

In [55]:
t = (1, 2, 3)

In [56]:
t[1]

2

In [57]:
id(t[1])

1820879968

In [58]:
id(t[2])

1820880000

In [59]:
t = ([1, 2], [3, 4])

In [60]:
id(t)

1721925260360

In [65]:
t[1].append(3)
t  #element is mutable

([1, 2], [3, 4, 3, 3, 3])

### Function Arguments and Mutability

In [66]:
# In python , Strings(str) are immutable objects

In [67]:
#Once a string has bee ncreated, the contents of the object cannot be changed

In [68]:
# In this code :
#my_var = 'hello'

#the only way to modify the "value" of my_var is to reassign myvar to another object

Immutable objects are Safe from unintended side effects!!!

In [70]:
def process(s):
    s = s + ' world'
    return s

In [71]:
my_var = "hello"

In [72]:
# We have two scopes here
#Module scope
  #myvar in module scope say  = "hello " (i.e points to an object "hello")
    

In [74]:
process(my_var)  # We are calling process function
# And passing my_var  i.e (MY_VAR 'S REFERENCE IS PASSED TO PROCESS!!!!')

'hello world'

In [76]:
#Now the Process's scope stores the reference in the variable "s"

# Now "s" points to the same object in the memory!

In [77]:
#Now the only way to get hello world, is to create a new object 
# with a different pointer to hello world!

In [78]:
print(my_var)

hello


In [79]:
#Now the function cannot change immutable objects 
# except for the tuples of lists example discussed before!

In [80]:
def process(lst):
    lst.append(100)

In [81]:
# We again have module and process scope!

In [82]:
my_list = [1, 2, 3]

In [84]:
process(my_list) # reference to list is passed into process

In [85]:
# Now lst also points to the same list
# Now we have appended 100 
# The memory address has not changed
# but the contents are changed!

In [87]:
my_list # Now its changed!
# This is a SIDE EFFECT!
#UNINTENDED SIDE EFFECTS!

[1, 2, 3, 100, 100]

#### Another variety is 
#### "immutable collection objects that contain mutable objects"

In [88]:
def process(t):
    t[0].append(999)

In [95]:
t = ([1, 2], 'a')

print(id(t)) 

1721925043272


In [94]:
process(t) # we modified the object!

In [93]:
t

([1, 2, 999], 'a')

In [97]:
print(id(t))  # Same id but the values are changed!

1721925043272


In [98]:
#Lets try again!

In [104]:
def process(s):
    print('initial s :: = {0}'.format(id(s)))
    s = s + 'world'
    print('Final s :: = {0}'.format(id(s)))

In [105]:
my_var = 'hello'

print('my var memory address = {0}'.format(id(my_var))) 

my var memory address = 1721927430192


In [108]:
process(my_var)  # Two different addresses!

initial s :: = 1721927430192
Final s :: = 1721927309680


In [107]:
print('my var memory address = {0}'.format(id(my_var))) # Hasnt changed!

my var memory address = 1721927430192


In [109]:
def changelist(lst):
    print('initial lst :: = {0}'.format(id(lst)))
    lst.append(1000)
    print('Final lst :: = {0}'.format(id(lst)))

In [111]:
l = [1, 2]
print(' l memory address = {0}'.format(id(l))) 

 l memory address = 1721926750088


In [113]:
changelist(l) # Same address!

initial lst :: = 1721926750088
Final lst :: = 1721926750088


In [115]:
print(' l memory address = {0}'.format(id(l)))  # Still the same!

 l memory address = 1721926750088


### Shared reference


In [116]:
# The term shared reference is the concept of two variables referencing the same
# object in the memory (i.e having the same memory address)

In [117]:
a = 10
b = a # copying memory address/references

In [118]:
# This may surprise you !


In [119]:
a = 10
b= 10

In [122]:
#b and a both point to he same object in the memory
# python's memory manager decides to reuse the memory references!
# TO perform some optimizations!

In [124]:
# Is this even safe?
# Yes 
# Because these objects are immutable!

In [125]:
a = [1, 2, 3]

In [127]:
b= a # Creates the shared reference!

In [129]:
b.append(100)

In [131]:
a # Now a's value has changed as well!

[1, 2, 3, 100]

In [132]:
# With mutable objects , the python memory manager will never create shared references
# Its never safe!

In [133]:
a = [1, 2, 3]
b = [1, 2, 3]

In [134]:
a == b

True

In [136]:
a is b # a and b are equal but not the same object!

False

In [137]:
a = "hello"
b = a

In [138]:
hex(id(a))

'0x190eadf7030'

In [139]:
hex(id(b))

'0x190eadf7030'

In [140]:
a = "hello"
b = "hello"

In [143]:
hex(id(a)) # Python memory manager did that 
# Why does it do that and when does it do that?

'0x190eadf7030'

In [142]:
hex(id(b))

'0x190eadf7030'

In [146]:
b = 'hello, world'

In [148]:
hex(id(b)) #Its memory address has changed!

'0x190eae097f0'

In [151]:
hex(id(a)) # "a" 's memory id has not changed!'

'0x190eadf7030'

In [152]:
a = [1, 2, 3]
b = a

In [153]:
hex(id(a))

'0x190eadf5ac8'

In [155]:
hex(id(b)) # We have a shared reference!

'0x190eadf5ac8'

In [156]:
b.append(15021)

In [157]:
# Since both a and b are pointing to the same object!

In [158]:
a

[1, 2, 3, 15021]

In [159]:
b

[1, 2, 3, 15021]

In [161]:
# Samething happens when you pass in a list to a function!
# As we can get unintended side effects!

In [162]:
a = 10
b = 10

In [163]:
hex(id(a))

'0x6c886d60'

In [164]:
hex(id(b))

'0x6c886d60'

In [165]:
a = 500
b = 500


In [166]:
hex(id(a))

'0x190eae0e3f0'

In [168]:
hex(id(b)) # Why is it different ? and when is the same and when is itdifferent?

'0x190eae0e430'

### Variable Equality!

In [1]:
# We can think of variable equality in two fundamental ways:
#I.e Memory address
# Or Object state (data)

In [ ]:
# To compare memory address we use "is"
# "is" is called identity operator whcih compares memory addresses

In [2]:
# If we want to compare values / data/ object state
# Then we have to use equality operator!

In [5]:
# "is not" : opposite of "is" operator

In [4]:
# " != "  is the opposite of " == " for the object state

In [6]:
# Examples

In [7]:
a = 10
b = a 

In [10]:
a is b       #shared reference

True

In [12]:
a == b      # in terms of values they are equal

True

In [ ]:
a = "hello"  
b = "hello"

In [13]:
a is b  

True

In [14]:
a == b

True

In [15]:
a = [1, 2, 3, 4]

b = [1, 2, 3, 4]

In [16]:
a is b

False

In [17]:
a == b

True

In [18]:
a = 10
b = 10.0

In [19]:
a is b

False

In [22]:
a == b # Why is this True??? Even though its not of same data type , its equal !!

True

### The None Object

In [23]:
# The None object can be assigned to variables to indicate that they are not set (in the way we would 
# expect them to be ), i.e an "empty" value (or null pointer)

In [24]:
# But the None object is a real object that is managed by the python memory manager

In [25]:
# Furthermore the memory manager will always use a shared reference when assigning a 
# variable to None

In [26]:
a = None
b = None
c = None

In [31]:
a is b

True

In [32]:
b is c

True

In [33]:
a is c

True

In [34]:
# SO WE CAN TEST IF A VARIABLE IS " NOT SET " or " EMPTY" by comparing its memory address to the 
# memory address of the NONE using the "is" operator

In [35]:
a is None

True

In [37]:
a == None

True

In [38]:
# None is not " Nothing ", its an actual object!!!

In [39]:
x = 10
x is None

False

In [40]:
x is not None

True

In [41]:
a =10
b = 10

In [43]:
id(a)

1468820832

In [44]:
id(b)

1468820832

In [45]:
a is b

True

In [46]:
print(' a ==b ', a == b)

 a ==b  True


In [47]:
a = 500
b = 500

In [48]:
id(a)

2159782138192

In [50]:
id(b) # Different address!

2159782138672

In [52]:
a is b # Not the same address

False

In [54]:
a ==  b

True

In [55]:
a = [1, 2, 3]
b = [1, 2, 3]

In [56]:
id(a)

2159782152968

In [57]:
id(b)

2159782152712

In [59]:
a == b  # Contents are equal!

True

In [60]:
a = 10

In [61]:
b = 10.0

In [64]:
a is b # int and float cant be the same object!

False

In [65]:
a == b # Same number!

True

In [66]:
a = 10 + 0j

In [67]:
type(a)

complex

In [68]:
type(b)

float

In [69]:
a is b

False

In [71]:
a == b # Still True!

True

In [72]:
id(None)

1468376208

In [74]:
type(None)  # So its a real object in memory!

NoneType

In [80]:
a = None
b = None
c = None

In [81]:
a is b

True

In [82]:
a is c

True

In [83]:
a is None

True

In [84]:
b is None

True

In [85]:
c is None

True

## Everything is an Object in Python

In [86]:
# One thing common with all  data types are 
# That they are instance of Classes ( aka they are objects!)

In [87]:
# Functions are instance of class function

In [88]:
# Classes ( not just instances, but the class itself!)

In [89]:
# Type eg type(10) itself is a class

In [90]:
# This means they all have a memory address!

* Meaning,Functions have memory address

In [91]:
#             def func():
#                       .....
#                        .....
     
#func is a variable that points to the function object!

In [92]:
# AS a Consequence:

* Any object can be assigned to a variable (including function)
* Any object can be passed to a function including functions.
* Any object can be returned from a function,  including functions


func is the name of the function


func() invokes the function! !

In [93]:
a = 10

In [96]:
print(type(a))  # a is an int instance

<class 'int'>


In [100]:
b = int(10)  # Classic Class notation!

In [101]:
b

10

In [102]:
print(type(b))

<class 'int'>


In [103]:
# Classes in python have inbuilt documentation!

In [104]:
help(int)

Help on class int in module builtins:

class int(object)
 |  int(x=0) -> integer
 |  int(x, base=10) -> integer
 |  
 |  Convert a number or string to an integer, or return 0 if no arguments
 |  are given.  If x is a number, return x.__int__().  For floating point
 |  numbers, this truncates towards zero.
 |  
 |  If x is not a number or if base is given, then x must be a string,
 |  bytes, or bytearray instance representing an integer literal in the
 |  given base.  The literal can be preceded by '+' or '-' and be surrounded
 |  by whitespace.  The base defaults to 10.  Valid bases are 0 and 2-36.
 |  Base 0 means to interpret the base from the string as an integer literal.
 |  >>> int('0b100', base=0)
 |  4
 |  
 |  Methods defined here:
 |  
 |  __abs__(self, /)
 |      abs(self)
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __and__(self, value, /)
 |      Return self&value.
 |  
 |  __bool__(self, /)
 |      self != 0
 |  
 |  __ceil__(...)
 |      Ceiling of

In [105]:
c = int()

In [106]:
c

0

In [107]:
c = int('101',base = 2)

In [108]:
c

5

In [109]:
def square (a):
    return a ** 2 

In [112]:
print(type(square))  # Its an instance of class function

<class 'function'>


In [114]:
f = square

In [120]:
f(2)  # invoking f!

4

In [116]:
id(square)

2159782330712

In [117]:
id(f)

2159782330712

In [118]:
f is square

True

In [119]:
f == square

True

In [121]:
f

<function __main__.square(a)>

In [122]:
# A function can also return a function

In [123]:
def cube (a):
    return a ** 3

In [124]:
def select_function(fn_id):
    if fn_id == 1:
        return square
    else:
        return cube

In [129]:
f = select_function(1)

In [131]:
f is square

True

In [132]:
f(2)

4

In [133]:
f = select_function(2)

In [135]:
f is cube

True

In [136]:
f(2) 

8

In [138]:
select_function(2)(3) #INVOKED CUBE !!!

27

In [139]:
# A fucntion cnabe passed to a function 

In [140]:
def exec_func(fn , n):
    #Apply fntion fn to n
    return fn(n)

In [141]:
exec_func(cube,3)

27

In [142]:
exec_func( square, 3)

9

In [143]:
# Functions are first class citizens!

## Python optimizations

In [145]:
# This is specific to python implementation
# This notebook is based on CPython
# Version of Cpython also matters!

In [147]:
# Interning 

In [148]:
# IT means Reusing objects  on demand!

In [149]:
# At sartup Pyrhon (Cpython), preloads (*caches) a global list of integers in range [-5, 256]
# Any time an intefer is referenced in the range, Python will use the cached version of that object

In [151]:
# Now
a = 5
b = 5
 
 # BOTH  a and b are between the range (-5 and 256) inclusive   
    
a is b

True

In [152]:
# But here

In [155]:
a = 500
b = 500
a is b  # 500 is outside this! so no shared reference!

False

The integers between range [-5, 256] are basically __Singleton Objects!__

* Singleton Objects are classes that can be instantiated only once , that's the reason for 
shared reference!

In [157]:
# The specific range [-5, 256] is chosen because small integers show up often!!

In [158]:
a = 10

In [159]:
# Python has to point to the existing reference for 10

In [160]:
#But if we write 
a = 257

In [161]:
#Python does not use the global list and creates it everytim1

### Examples!

In [162]:
a = 10
b = 10

In [163]:
a is b


True

In [165]:
print(id(a), id(b))

1468820832 1468820832


In [166]:
a = -5
b = -5

In [168]:
print(id(a), id(b))  # Memory addresses are the same

1468820352 1468820352


In [169]:
a is b

True

In [171]:
a = 256
b = 256

In [172]:
a is b

True

In [173]:
a = 257
b = 257
a is b

False

In [183]:
a = 10
b = int(10)
c = int('10')
d = int('1010',2)

In [184]:
a is b

True

In [185]:
b is c

True

In [186]:
a is c

True

In [187]:
a is d

True

In [188]:
print(id(a), id(a), id(b), id(c), id(d))

1468820832 1468820832 1468820832 1468820832 1468820832


In [190]:

# Memory addresses are the same!

### Python Optimization String interning 

In [191]:
# Some strings are also automatically interned - BUT NOT ALL

* As the Python code is compiled, __identifiers__ are interend,

__BUT DONT COUNT ON IT!!__

__wHY DOES PYTHON DO THIS? __

iTS BECAUSE OF OPTIMIZATION BY PYTHON

In [192]:
# Not all strings are interened by Python

In [193]:
# BUT YOU CAN FORCE ISTRINGS TO BE INTERNED BY USING THE sys,intern()

In [194]:
# Eg:
# import sys
# a = sys.intern('the quick fox')
# b = sys.intern('the quick  reddddddddddddd fox')#

In [195]:
# When should you do this?
# Dealing with a large number ofstrings that could have high repition
# Tokeninzing a large corpus of text (NLP)

#Memory optimization!
# And help speeds up comparisions

In [196]:
# In general dont start interening without necessity!

In [202]:
a = "hello"
b = "hello"

In [203]:
print(id(a), id(b))

2159782857952 2159782857952


In [214]:
a is b

True

In [216]:
a = ' hello world'
b = 'hello world'

In [217]:
print(id(a), id(b))

2159782850288 2159782875184


In [218]:
a is b

False

In [206]:
# DONT COUNT ON INTERENING TO HAPPEN!

In [210]:
a = "hello"
b = "hello"

In [211]:
a == b

True

In [212]:
a is b

True

In [219]:
a = '_this_is_a_long_string_that_could_be_used_as_an_identifier'

In [220]:
b = '_this_is_a_long_string_that_could_be_used_as_an_identifier'

In [221]:
a == b

True

In [223]:
a is b  # Even for a long string if 
# it looks like an identifier it gets interened!

True

### Forcing string interening


In [224]:
import sys

In [225]:
a = sys.intern('hello world')

In [226]:
b = sys.intern('hello world')

In [227]:
c  = 'hello world'

In [229]:
print(id(a), id(b), id(c))

# ID OF  " a" and " b " is the same 
# PYthon does not intern C 
## Look!!

2159782875440 2159782875440 2159782876272


In [231]:
%timeit a == b

29.3 ns ± 0.214 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [232]:
%timeit a == c

34.5 ns ± 0.119 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [234]:
# More time!
# Reduces time !

In [235]:
def compare_using_equals(n):
    a = 'a long string that is not interned' * 200
    b = 'a long string that is not interned' * 200
    for i in range(n):
        if a ==b:
            pass
        

In [236]:
def compare_using_interning(n):
    a = sys.intern('a long string that is not interned' * 200)
    b =  sys.intern('a long string that is not interned' * 200)
    for i in range(n):
        if a is b:
            pass


In [241]:
import time
start = time.perf_counter()

compare_using_equals(10000000)

end = time.perf_counter()

print(f'This for the EQUALITY :', end - start)

This for the EQUALITY : 2.4757142124768734


In [243]:
import time
start = time.perf_counter()

compare_using_interning(100000)

end = time.perf_counter()

print(f'This for the EQUALITY :', end - start)

# LOOOL  2.3 milliseconds!

This for the EQUALITY : 0.0022987099110309828


#### Python Optimization Peephole

In [2]:
# This happens at compile time!

In [3]:
# Constant expressions
# Numeric calculations   eg : 24 *60
# It might calculate 24 * 60 again and again  ... 
# Python will actually pre-calculate 24 * 60 -> 1440
# Python will do that for us

In [5]:
# Short sequences of length < 20
# (1, 2) *5   is precalculated and gets stored!
# Similarly  'hello ' + ' world'

In [6]:
# but not  'the quick brown fox ' * 10 is more than 20 characters !

In [7]:
# Membership tests : mutablis are replaced by immutables

In [10]:
#when membership tests such as 
# if e in [1, , 2, 3]:  # here [1, 2, 3] is a constanct, so python 
# replaces list [1, 2, 3] to its immutable form (1, 2, 3) i.e tuples 
# and sets are converted to Frozen sets!

In [11]:
# Set membership is faster than list or tuple memberships { sets are basically dictionaries}

In [12]:
#So instead of writing :
#if e in [1, 5, 6] : 

#write if e in {1, 5, 6}

In [13]:
def f():
    a = 24 *100
    b = (1, 2) *5
    c = 'abc' * 3
    c= 'ab' * 11
    e = 'the quick brown fox'*5
    f = ['a','b'] * 3

In [17]:
f.__code__.co_consts
# (1,2) * 5 is precalculated!

(None,
 24,
 100,
 1,
 2,
 5,
 'abc',
 3,
 'ab',
 11,
 'the quick brown fox',
 'a',
 'b',
 2400,
 (1, 2),
 (1, 2, 1, 2, 1, 2, 1, 2, 1, 2),
 'abcabcabc')

In [18]:
def f1(e):
    if e in [1,2,3]:
        pass

In [20]:
f1.__code__.co_consts
# list [1, 2, 3] became a tuple (1, 2, 3)

(None, 1, 2, 3, (1, 2, 3))

In [21]:
def f1(e):
    if e in {1,2,3}:
        pass

In [23]:
f1.__code__.co_consts
#set becomes frozen sets!

(None, 1, 2, 3, frozenset({1, 2, 3}))

In [24]:
import string
import time

In [25]:
string.ascii_letters

'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [26]:
char_list = list(string.ascii_letters)

In [27]:
char_tuple = tuple(string.ascii_letters)

In [28]:
char_set = set(string.ascii_letters)

In [36]:
print(char_list)  # Maintains the order as in the string!

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


In [37]:
print(char_tuple) # Maintains the order as in the string!

('a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z')


In [38]:
print(char_set)  # No order

{'S', 'f', 'l', 'p', 'T', 'a', 'w', 'M', 'C', 'O', 'o', 'L', 'd', 't', 'c', 'z', 'B', 'V', 'k', 'N', 'K', 'E', 'n', 'y', 'i', 'W', 'u', 'P', 'q', 'r', 'A', 'U', 'F', 'R', 's', 'I', 'x', 'b', 'g', 'h', 'm', 'j', 'Q', 'v', 'H', 'X', 'Z', 'D', 'G', 'Y', 'J', 'e'}


In [39]:
def membership_test(n, container):
    for i in range(n):
        if 'z'  in container:
            pass

In [40]:
start = time.perf_counter()
membership_test(10000000, char_list)
end = time.perf_counter()
print('list: ', end - start)

list:  2.7123143047560143


In [ ]:
start = time.perf_counter()
membership_test(10000000, char_tuple)
end = time.perf_counter()
print('tuple: ', end - start)